# Putting things together 2

Here we will define a cell model with multiple sections and create a network model.

## Planning how to wire cells

In [ ]:
%%bash 
pip install networkx

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

We begin with how many cells are in the network.

In [ ]:
Ncells = 500                    # Number of cells
Nexc = int((Ncells/5)*4)         # Excitatory cells = 80%
Ninh = int(Ncells/5)             # Inhibitory cells = 20%
nexcpre = int(Nexc*0.1)          # Presynaptic excitatory cells for each neuron = 10% of excitatory cells
ninhpre = int(Ninh*0.1)          # Presynaptic inhibitory cells for each neuron = 10% of inhibitory cells

print('Ncells =', Ncells, '\nNexc =', Nexc, '\nNinh =', Ninh, '\nnexcpre =',nexcpre, '\nninhpre =', ninhpre)

Here we first specify an anatomical basis of the network, which is defined by a directional graph structure, containing nodes for cells and edges for cell-to-cell connections.

In [ ]:
G = nx.DiGraph()

In [ ]:
for i in range(Ncells):
    if i<Nexc:
        G.add_node(i, cell_type='E', rho=200, c=10)
    else:
        G.add_node(i, cell_type='I', rho=80, c=10)

In [ ]:
for i in range(Ncells):
    
    # Choose nexcpre cells from Nexc excitatory cells
    exc_pre = np.random.randint(0, Nexc, nexcpre)
    # Choose ninhpre cells from Ninh inhibitory cells
    inh_pre = np.random.randint(Nexc, Ncells, ninhpre)

    for k in exc_pre:
        if i!=k:  # No self-connection
            # pnm.nc_append(id_of_presyn_cell, id_of_postsyn_cell, synapse_id, syn_weight, propagation_delay, threshold)
            if k in G[i]:
                G[i][k]['weight'] +=1
            else:
                G.add_edge(i, k, weight=1)
            
    for k in inh_pre:
        if i!=k:  # No self-connection
            if k in G[i]:
                G[i][k]['weight'] +=1
            else:
                G.add_edge(i, k, weight=1)

Let's check the nodes and connections 

In [ ]:
for i in G.nodes:
    print(i, G.nodes[i]['cell_type'])

In [ ]:
i = 0
for k in G[i]:
    print(k, G.nodes[k]['cell_type'], G[i][k]['weight'], G[i][k]['target'])

We can save the constructed network:

In [ ]:
import pickle
with open('balanced_Mainen.pkl', 'wb') as f:
    pickle.dump(G, f)

Then, the network can be reused later.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

import pickle
with open('balanced_Mainen.pkl', 'rb') as f:
    G = pickle.load(f)

# i = 0
# for k in G[i]:
#     print(k, G.nodes[k]['cell_type'], G[i][k]['weight'], G[i][k]['target'])

## Building a network model after the plan

We first create neurons in the network according to the plan:

In [ ]:
from neuron import h, gui, load_mechanisms
from net_manager import SerialNetManager
from cell_models import Mainen

pnm = SerialNetManager(len(G.nodes))

for i in G.nodes:
    pnm.register_cell(i, Mainen(i, rho=G.nodes[i]['rho'], c=G.nodes[i]['c']))

Then, we add brief stimuli for the cells to kickstart the network activity

In [ ]:
# Kickstart stims
ext_stims = []
ext_ncs = []
for i in G.nodes:
    stim = h.NetStimFD(pnm.gid2cell[i].soma(0.5))
    stim.interval = 3
    stim.noise = 1
    stim.start = 0
    stim.duration = 7
    stim.seed(i+1223)

    nc = h.NetCon(stim, pnm.gid2cell[i].synlist[1])
    nc.weight[0] = 2.5e-3
    ext_stims.append((stim, nc))

In [ ]:
pnm.want_all_spikes()

h.tstop = 400
h.init()
pnm.run()

In [ ]:
Ncells = len(G.nodes)

fig, ax = plt.subplots(figsize=(20, 10))
ax.plot(pnm.spikevec, pnm.idvec, '.k')
ax.set(xlim=[0, 200], ylim=[0, Ncells])
ax.set(xlabel='Time (ms)', ylabel='Neuron')

Then we wire the cells for each edge in our plan.

In [ ]:
gexc = 0.5e-3     # initial value --- will be adjusted later
ginh = 10.*gexc   # initial value --- will be adjusted later

# Reset all NetCon's
pnm.nc_reset()

# Go around every cell
for i in G.nodes:
    for k in G[i]:
        t_delay = np.random.rand()
        if G.nodes[k]['cell_type']=='E':
            g = gexc*G[i][k]['weight']
            pnm.nc_append(k, i, 0, g, t_delay, thresh=-10)
            pnm.nc_append(k, i, 1, g, t_delay, thresh=-10)
        else:
            g = ginh*G[i][k]['weight']
            pnm.nc_append(k, i, 2, g, t_delay, thresh=-10)

Here are functions to adjuct synaptic conductances and pack the simulation results into a matrix:

In [ ]:
def set_global_synaptic_conductance(gexc, ginh, G, pnm):
    for nc in pnm.netcons:
        k, i, _ = nc
        if G.nodes[k]['cell_type']=='E':
            g = gexc*G[i][k]['weight']
        else:
            g = ginh*G[i][k]['weight']
        pnm.netcons[nc].weight[0] = g

        
def repack_result(pnm, G, t_init=75):
    x = np.array([pnm.spikevec, pnm.idvec]).T
    x = x[x[:,0]>t_init,:]
    x[:,0] -= t_init
    e_or_i = np.array([G.nodes[n]['cell_type']=='E' for n in G.nodes])
    e_or_i = e_or_i[x[:,1].astype(int)]
    x = np.hstack([x, e_or_i[:,None]])
    return x

In [ ]:
t_init = 75

h.tstop = t_init+400
h.init()
pnm.run()

In [ ]:
x = repack_result(pnm, G)

col = np.zeros((x.shape[0], 3));
col[x[:,2]==0,:] = [0.5,0,0]

fig, ax = plt.subplots(figsize=(15, 5))
ax.scatter(x[:,0], x[:,1], 2, c=col)
ax.set(xlabel='Time (ms)', ylabel='Neuron', xlim=[0, h.tstop-75])

Let's adjust the synaptic parameters until we get stable asynchronous activity:

In [ ]:
gexc=1.6e-3
set_global_synaptic_conductance(gexc, 150.*gexc, G, pnm)

Let's make a firing rate histogram averaged over all the cells first

In [ ]:
tstop = h.tstop-t_init
tc = np.arange(tstop+1)
spikecount_all, _ = np.histogram(x[:,0], tc)
t = tc[:-1]

_, ax = plt.subplots(figsize=(20, 5))
ax.step(t, spikecount_all/Ncells*1e3, 'k')
ax.set(xlim=[0, tstop], xlabel='time (ms)', ylabel='rate (Hz)')

We do the same for the excitatory and inhibitory population

In [ ]:
tc = np.arange(tstop+1)
spikecount_exc, _ = np.histogram(x[x[:,2]==1,0], tc)
spikecount_inh, _ = np.histogram(x[x[:,2]==0,0], tc)
t = tc[:-1]

Nexc = len([G.nodes[i] for i in G.nodes if G.nodes[i]['cell_type']=='E'])
Ninh = len([G.nodes[i] for i in G.nodes if G.nodes[i]['cell_type']=='I'])

_, axs = plt.subplots(figsize=(20, 5), nrows=2, sharex=True)
axs[0].step(t, spikecount_exc/Nexc*1e3, 'k')
axs[1].step(t, spikecount_inh/Ninh*1e3, 'g')
axs[1].set(xlim=[0, 500], xlabel='time (ms)', ylabel='rate (Hz)')
axs[0].set(ylabel='rate (Hz)')

Note that the E/I population activities are positively correlated.

In [ ]:
np.corrcoef(spikecount_exc, spikecount_inh)